In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.vllm import start_vllm

vllm = await start_vllm(
    model="NousResearch/Hermes-2-Theta-Llama-3-8B",
    env={"VLLM_ALLOW_LONG_MAX_MODEL_LEN": "1"},
    block_size=32,
    disable_log_requests=True,
    enable_prefix_caching=True,
    enforce_eager=True,
    gpu_memory_utilization=0.9,
    max_model_len=16384,
    max_num_seqs=512,
    max_num_batched_tokens=16384 * 4,
    return_tokens_as_token_ids=True,
    swap_space=32,
    logits_processor_pattern=r"vllm\..*"
)

$ vllm serve NousResearch/Hermes-2-Theta-Llama-3-8B --block-size=32 --disable-log-requests --enable-prefix-caching --enforce-eager --gpu-memory-utilization=0.9 --max-model-len=16384 --max-num-seqs=512 --max-num-batched-tokens=65536 --return-tokens-as-token-ids --swap-space=32 --logits-processor-pattern=vllm\..* --port=8001 --api-key=default
INFO 12-13 18:34:24 api_server.py:634] vLLM API server version 0.1.dev3782+gf9e303a
INFO 12-13 18:34:24 api_server.py:635] args: Namespace(subparser='serve', model_tag='NousResearch/Hermes-2-Theta-Llama-3-8B', config='', host=None, port=8001, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key='default', lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=True, disable_front

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.10it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.54it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.22it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.25it/s]



INFO 12-13 18:34:43 model_runner.py:1096] Loading model weights took 14.9595 GB
INFO 12-13 18:34:45 worker.py:237] Memory profiling results: duration=2.61 seconds, total_gpu_memory=79.10GiB, initial_memory_usage=15.50GiB, peak_torch_memory=21.74GiB, memory_usage_post_profile=15.60GiB, non_torch_memory=0.61GiB, kv_cache_size=48.84GiB, gpu_memory_utilization=0.90.
INFO 12-13 18:34:45 gpu_executor.py:76] # GPU blocks: 12502, # CPU blocks: 8192
INFO 12-13 18:34:45 gpu_executor.py:80] Maximum concurrency for 16384 tokens per request: 24.42x
INFO 12-13 18:34:56 llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 13.41 seconds
INFO 12-13 18:34:57 api_server.py:569] Using supplied chat template:
INFO 12-13 18:34:57 api_server.py:569] None
INFO 12-13 18:34:57 launcher.py:19] Available routes are:
INFO 12-13 18:34:57 launcher.py:27] Route: /openapi.json, Methods: GET, HEAD
INFO 12-13 18:34:57 launcher.py:27] Route: /docs, Methods: GET, HEAD
INFO 12-13 18:34:57 launcher.p

INFO:     Started server process [39240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO 12-13 18:35:00 chat_utils.py:331] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
INFO:     127.0.0.1:42410 - "POST /v1/chat/completions HTTP/1.1" 200 OK
vLLM server started succesfully. Logs can be found at ./logs/vllm.log


In [7]:
vllm.process.terminate()

[autoreload of vllm.entrypoints.openai.protocol failed: Traceback (most recent call last):
  File "/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 330, in update_class
    old_obj = getattr(old, key)
              ^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/pydantic/_internal/_model_construction.py", line 262, in __getattr__
    raise AttributeError(item)
AttributeError: max_tokens
]


In [4]:
from lib.utils import black_print

black_print(
    await vllm.client.chat.completions.create(
        messages=[{"role": "user", "content": "Hi"}],
        model="NousResearch/Hermes-2-Theta-Llama-3-8B",
        max_tokens=1,
        extra_body={
            "logits_processors": [
                {
                    "qualname": "vllm.entrypoints.openai.logits_processors.AllowedTokenIdsLogitsProcessor",
                    "args": [[22222]],
                    # "kwargs": {"allowed_ids": [22222]},
                },
                # "atreides.experiments.lib.logits_processors.random_logits_processor",
            ],
        },
    )
)

ChatCompletion(
    id="chatcmpl-38b9c68fa00b4834b222f7d246e483a1",
    choices=[
        Choice(
            finish_reason="length",
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content="curacy",
                refusal=None,
                role="assistant",
                audio=None,
                function_call=None,
                tool_calls=[],
            ),
            stop_reason=None,
        )
    ],
    created=1734114925,
    model="NousResearch/Hermes-2-Theta-Llama-3-8B",
    object="chat.completion",
    service_tier=None,
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=1,
        prompt_tokens=10,
        total_tokens=11,
        completion_tokens_details=None,
        prompt_tokens_details=None,
    ),
    prompt_logprobs=None,
)
